In [1]:
import numpy as np
import torch
import random
seed=0
np.random.seed(seed)
torch.manual_seed(seed)
random.seed(seed)

# load data

In [2]:
from glob import glob
import os
images=[]
labels=[]
for folder in os.listdir('../dataset/wrist_xray'):
    for pic in glob('../dataset/wrist_xray/{}/*.jpg'.format(folder)):
        images.append(pic)
        labels.append(folder)
        

In [3]:
import pandas as pd
data=pd.DataFrame(zip(images,labels),columns=['img','label'])
data.label=data.label.map({'Fracture':0,'Normal':1})
data = data.sample(frac=1).reset_index(drop=True)
data.head()

,img,label
0,../dataset/wrist_xray/Normal\1.jpg,1
1,../dataset/wrist_xray/Normal\30.jpg,1
2,../dataset/wrist_xray/Fracture\47 Male (A View...,0
3,../dataset/wrist_xray/Normal\20.jpg,1
4,../dataset/wrist_xray/Normal\52.jpg,1


# top 5 lightest models in torchvision
1	squeezenet1_1	1235496  
2	shufflenet_v2_x0_5	1366792  
3	mnasnet0_5	2218512  
4	mobilenet_v2	3504872  
5	densenet121	7978856  `

In [4]:
import sys
sys.path.append("../src") 

In [5]:
from models.squeezenet import squeezenet1_1
from models.mymodels import mymodel1,mymodel3
from models.shufflenet import shufflenet_v2_x0_5
from models.mnastnet import mnasnet0_5
from models.densenet import densenet121
from models.mobilenet import mobilenet
from cross_vals import kfoldcv

In [6]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)/1e6

In [7]:
print('MyNet parameters :' ,count_parameters(mymodel1()))
print('SqueezeNet parameters :', count_parameters(squeezenet1_1()))
print('ShuffleNet parameters :', count_parameters(shufflenet_v2_x0_5()))
print('MnastNet parameters :', count_parameters(mnasnet0_5()))
print('MobileNet parameters :', count_parameters(mobilenet()))
print('DenseNet parameters :', count_parameters(densenet121()))


MyNet parameters : 1.158232
SqueezeNet parameters : 1.234344
ShuffleNet parameters : 1.36636
MnastNet parameters : 2.218224
MobileNet parameters : 3.504296
DenseNet parameters : 7.972584


In [8]:
epoch=32
batchsize=8
lr=0.001
skip_tuning=True

dim='gray' for gray scale images  
skip_tuning=False, no grid search applied for hyper-parameter tuning,  
aug=1-> pytorch augmentation  
aug=0-> no augmentation  
aug=2-> albumentation augmentation    
aug=3-> rand augmentation

# OurNet

In [9]:
%%time
train_cv,clf_report=kfoldcv(model=mymodel3(),epochs=epoch,batchsize=batchsize,data=data,lr=lr,skip_tuning=skip_tuning,aug=1)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.822   0.796     0.806
Class 1       0.730   0.772     0.746
macro         0.776   0.782     0.776
weighted      0.792   0.780     0.784
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.850   0.842     0.844
Class 1       0.796   0.818     0.802
macro         0.824   0.830     0.822
weighted      0.832   0.828     0.828
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.858   0.792     0.822
Class 1       0.744   0.840     0.786
macro         0.802   0.816     0.804
weighted      0.818   0.808     0.810
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.854   0.808     0.828
Class 1       0.762   0.834     0.790
macro         0.806   0.820     0.808
weighted      0.822   0.812     0.816
--------------------------Accuracy Tab

,DT,SVM,LR,KNN
0,0.77,0.77,0.77,0.72
1,0.74,0.82,0.87,0.82
2,0.79,0.90,0.82,0.87
3,0.76,0.76,0.74,0.76
4,0.84,0.89,0.84,0.89
mean,0.78,0.828,0.808,0.812


# SqueezeNet

In [10]:
%%time
train_cv,clf_report=kfoldcv(model=squeezenet1_1(),epochs=epoch,batchsize=batchsize,data=data,lr=lr,skip_tuning=skip_tuning,aug=1)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

d:\datasets\image dataset\siamese\venv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\datasets\image dataset\siamese\venv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\datasets\image dataset\siamese\venv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.644   0.598     0.614
Class 1       0.510   0.562     0.526
macro         0.578   0.578     0.568
weighted      0.600   0.574     0.578
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.612   0.874     0.712
Class 1       0.300   0.206     0.238
macro         0.456   0.540     0.474
weighted      0.482   0.596     0.514
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.630   0.632     0.626
Class 1       0.486   0.486     0.478
macro         0.560   0.560     0.554
weighted      0.582   0.568     0.568
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.640   0.672     0.652
Class 1       0.542   0.502     0.516
macro         0.592   0.586     0.584
weighted      0.616   0.600     0.604
--------------------------Accuracy Tab

,DT,SVM,LR,KNN
0,0.74,0.74,0.82,0.79
1,0.41,0.44,0.44,0.46
2,0.56,0.74,0.56,0.64
3,0.53,0.53,0.55,0.58
4,0.63,0.53,0.47,0.53
mean,0.574,0.596,0.568,0.6


# ShuffleNet

In [11]:
%%time
train_cv,clf_report=kfoldcv(model=shufflenet_v2_x0_5(),epochs=epoch,batchsize=batchsize,data=data,lr=lr,skip_tuning=skip_tuning,aug=1)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

d:\datasets\image dataset\siamese\venv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\datasets\image dataset\siamese\venv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\datasets\image dataset\siamese\venv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\datasets\i

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.614   0.676     0.628
Class 1       0.506   0.434     0.440
macro         0.562   0.552     0.538
weighted      0.588   0.552     0.550
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.582   0.912     0.696
Class 1       0.176   0.102     0.106
macro         0.378   0.508     0.400
weighted      0.408   0.548     0.436
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.584   0.620     0.596
Class 1       0.448   0.416     0.418
macro         0.516   0.518     0.508
weighted      0.538   0.530     0.524
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.580   0.646     0.604
Class 1       0.430   0.378     0.386
macro         0.504   0.514     0.494
weighted      0.524   0.522     0.510
--------------------------Accuracy Tab

,DT,SVM,LR,KNN
0,0.69,0.49,0.49,0.46
1,0.38,0.59,0.56,0.41
2,0.59,0.64,0.62,0.69
3,0.55,0.47,0.53,0.47
4,0.55,0.55,0.45,0.58
mean,0.552,0.548,0.53,0.522


# Mnasnet

In [12]:
%%time
train_cv,clf_report=kfoldcv(model=mnasnet0_5(),epochs=epoch,batchsize=batchsize,data=data,lr=lr,skip_tuning=skip_tuning,aug=1)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

d:\datasets\image dataset\siamese\venv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\datasets\image dataset\siamese\venv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\datasets\image dataset\siamese\venv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.626   0.630     0.616
Class 1       0.510   0.504     0.486
macro         0.568   0.566     0.552
weighted      0.592   0.564     0.562
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.618   0.878     0.708
Class 1       0.412   0.246     0.258
macro         0.514   0.562     0.482
weighted      0.530   0.586     0.502
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.622   0.634     0.616
Class 1       0.492   0.470     0.462
macro         0.556   0.554     0.542
weighted      0.580   0.556     0.554
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.606   0.630     0.612
Class 1       0.476   0.448     0.450
macro         0.542   0.540     0.532
weighted      0.562   0.560     0.552
--------------------------Accuracy Tab

,DT,SVM,LR,KNN
0,0.59,0.49,0.49,0.54
1,0.51,0.64,0.62,0.56
2,0.62,0.72,0.46,0.67
3,0.55,0.53,0.55,0.53
4,0.55,0.55,0.66,0.50
mean,0.564,0.586,0.556,0.56


# MobileNet

In [13]:
%%time
train_cv,clf_report=kfoldcv(model=mobilenet(),epochs=epoch,batchsize=batchsize,data=data,lr=lr,skip_tuning=skip_tuning,aug=1)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.608   0.570     0.584
Class 1       0.464   0.504     0.476
macro         0.536   0.536     0.530
weighted      0.558   0.538     0.544
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.584   0.886     0.700
Class 1       0.446   0.146     0.212
macro         0.514   0.516     0.454
weighted      0.544   0.570     0.496
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.620   0.664     0.636
Class 1       0.486   0.452     0.458
macro         0.554   0.556     0.546
weighted      0.578   0.564     0.564
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.568   0.678     0.610
Class 1       0.456   0.318     0.360
macro         0.510   0.500     0.484
weighted      0.536   0.528     0.512
--------------------------Accuracy Tab

,DT,SVM,LR,KNN
0,0.54,0.54,0.51,0.56
1,0.64,0.56,0.51,0.54
2,0.51,0.64,0.59,0.49
3,0.47,0.50,0.63,0.37
4,0.53,0.61,0.58,0.68
mean,0.538,0.57,0.564,0.528


# DenseNet121

In [14]:
%%time
train_cv,clf_report=kfoldcv(model=densenet121(),epochs=epoch,batchsize=batchsize,data=data,lr=lr,skip_tuning=skip_tuning,aug=1)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.628   0.590     0.608
Class 1       0.478   0.516     0.496
macro         0.556   0.556     0.554
weighted      0.580   0.570     0.574
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.714   0.762     0.734
Class 1       0.648   0.572     0.602
macro         0.684   0.670     0.670
weighted      0.692   0.694     0.686
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.742   0.738     0.734
Class 1       0.648   0.660     0.642
macro         0.698   0.702     0.690
weighted      0.718   0.696     0.698
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.686   0.670     0.676
Class 1       0.558   0.564     0.556
macro         0.622   0.618     0.614
weighted      0.644   0.632     0.632
--------------------------Accuracy Tab

,DT,SVM,LR,KNN
0,0.62,0.67,0.69,0.62
1,0.56,0.74,0.62,0.69
2,0.56,0.77,0.69,0.51
3,0.45,0.68,0.74,0.71
4,0.66,0.61,0.74,0.63
mean,0.57,0.694,0.696,0.632
